In [2]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [3]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [4]:
data =[]
for i in range(0,1):
    txt1 = open('태종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '태종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'태종실록'))

In [5]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("태종실록"," "))
df=df.map(lambda x : x.replace("태종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [6]:
len(df)

855

In [7]:
okt = Okt()

In [8]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','이제','하니','반드시','모두'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [9]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [10]:
 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [11]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	백성	0.018228178843855858
	가지	0.007067200727760792
	영락	0.0070346202701330185
	의논	0.0054397061467170715
	하소	0.005153974983841181
	자가	0.004637149162590504
	만일	0.004249207675457001
	수령	0.004052168224006891
	소서	0.003973518498241901
	원컨대	0.003949370700865984


Topic ID: 1
	백성	0.015394079498946667
	영락	0.009742222726345062
	태백성	0.006073309108614922
	자가	0.005808726418763399
	만일	0.0054992800578475
	어찌	0.005438097286969423
	생각	0.005120919086039066
	비록	0.004961246624588966
	다시	0.004793557338416576
	보고	0.004701169207692146


Topic ID: 2
	백성	0.01232222467660904
	영락	0.008120142854750156
	생각	0.005584674887359142
	가지	0.005425611510872841
	의논	0.005106894765049219
	다시	0.004776043351739645
	시행	0.004173831082880497
	어찌	0.004164020996540785
	비록	0.003998587839305401
	우리	0.0038521019741892815


Topic ID: 3
	백성	0.011698641814291477
	어찌	0.009100708179175854
	영락	0.008428768254816532
	의논	0.006702114827930927
	자가	0.006279387045651674
	상소	0.0061880191788077354
	다시	0.0059026740491

In [12]:
# 백성
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.025754 -0.001479       1        1  12.414034
4     -0.011860  0.007264       2        1  10.719824
15     0.016922 -0.001969       3        1   8.761265
16     0.025552  0.041814       4        1   8.313708
0      0.008069 -0.029081       5        1   7.560193
7      0.005087  0.000598       6        1   6.679852
5     -0.028037 -0.037655       7        1   6.652254
3     -0.012235  0.028532       8        1   5.223020
17     0.000697  0.037538       9        1   4.383055
19     0.013381 -0.000897      10        1   4.077677
12    -0.023751  0.004244      11        1   3.688370
9      0.000483  0.016884      12        1   3.201346
13    -0.032543 -0.001689      13        1   3.026253
18     0.056844 -0.037329      14        1   2.893070
10    -0.043229 -0.019812      15        1   2.771094
2      0.024005 -0.005847      16        1   2.128752
1      0.007961 -0.027147      17        1   2.120300
8     -0.016875  0.008620      18        1   2.085565
6     -0.027643 -0.005278      19        1   1.933214
11     0.011420  0.022689      20        1   1.367163, topic_info=    Term         Freq        Total Category  logprob  loglift
12    백성  1455.000000  1455.000000  Default  30.0000  30.0000
21    영락   694.000000   694.000000  Default  29.0000  29.0000
807  태백성   168.000000   168.000000  Default  28.0000  28.0000
136   어찌   538.000000   538.000000  Default  27.0000  27.0000
17    수령   279.000000   279.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
8     대간     2.964958   132.855118  Topic20  -5.9644   0.4900
23    자가     3.625446   410.874268  Topic20  -5.7633  -0.4379
204   폐단     3.246071   240.991913  Topic20  -5.8738  -0.0149
560   상서     2.633313   100.586823  Topic20  -6.0830   0.6497
648   의논     2.841079   425.001923  Topic20  -6.0071  -0.7155

[1563 rows x 6 columns], token_table=      Topic      Freq Term
term                      
5770      2  0.618173   가권
7136      4  0.648587   가렴
7504      7  0.433845   가마
7504     14  0.433845   가마
885       1  0.067435   가뭄
...     ...       ...  ...
2980      4  0.360158   후손
2980      5  0.144063   후손
2980      6  0.072032   후손
2980      7  0.072032   후손
6925      5  0.662599  흑산도

[5040 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 5, 16, 17, 1, 8, 6, 4, 18, 20, 13, 10, 14, 19, 11, 3, 2, 9, 7, 12])

In [13]:
# 주제 : 제사

In [14]:
data =[]
for i in range(0,1):
    txt1 = open('태종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '태종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '제사' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'태종실록'))

In [15]:
len(df)

855

In [16]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("태종실록"," "))
df=df.map(lambda x : x.replace("태종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [19]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','하니''모두'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [20]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [21]:
 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [22]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	제사	0.009160848334431648
	하니	0.00862882100045681
	영락	0.008055564947426319
	모두	0.0077057876624166965
	어찌	0.006752719171345234
	사냥	0.006661408115178347
	비록	0.006531725637614727
	예조	0.005959650035947561
	반드시	0.005830070469528437
	의정부	0.004651475697755814


Topic ID: 1
	제사	0.012930396944284439
	모두	0.010915699414908886
	영락	0.010273109190165997
	하니	0.006123322527855635
	이제	0.005104808136820793
	그대로	0.004930918104946613
	예조	0.004794480744749308
	종이	0.004346061032265425
	태조	0.004259440582245588
	반드시	0.0042076134122908115


Topic ID: 2
	제사	0.03560057282447815
	영락	0.014572059735655785
	모두	0.010325858369469643
	하니	0.008023177273571491
	예조	0.006595340557396412
	의정부	0.006077223923057318
	관찰사	0.005942807532846928
	의논	0.005350931081920862
	절제	0.005186355207115412
	하소	0.005128149408847094


Topic ID: 3
	제사	0.013571378774940968
	영락	0.00910460390150547
	모두	0.007532876916229725
	절제	0.006319186184555292
	하니	0.005672143306583166
	때문	0.005044639576226473
	도절	0.00503780646

In [23]:

pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.008249  0.031164       1        1  28.581518
0      0.019598 -0.015631       2        1   6.516034
3      0.019059 -0.023060       3        1   6.381302
13    -0.012331  0.026015       4        1   5.768603
6     -0.000344  0.027812       5        1   5.176121
15    -0.036085  0.003460       6        1   4.985122
4      0.027407  0.022785       7        1   4.883811
2     -0.044476  0.051374       8        1   4.852895
5     -0.024144  0.007943       9        1   4.060658
18     0.009328  0.000227      10        1   4.015602
7     -0.069700  0.003052      11        1   3.888988
10     0.037981  0.025311      12        1   3.294101
19    -0.003878 -0.011864      13        1   2.914402
16    -0.032612 -0.081807      14        1   2.672736
17     0.027389  0.015821      15        1   2.620136
8      0.029392 -0.014445      16        1   2.298132
11     0.036875 -0.013588      17        1   2.268630
1      0.021964 -0.000969      18        1   1.864228
14     0.013393 -0.039682      19        1   1.826290
9     -0.027065 -0.013918      20        1   1.130697, topic_info=    Term         Freq        Total Category  logprob  loglift
7     제사  1158.000000  1158.000000  Default  30.0000  30.0000
4     영락   612.000000   612.000000  Default  29.0000  29.0000
119   모두   433.000000   433.000000  Default  28.0000  28.0000
46    절제   181.000000   181.000000  Default  27.0000  27.0000
23    도절   177.000000   177.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
679   우리     1.640220   191.340057  Topic20  -5.8374  -0.2769
45   전라도     1.494445   121.197670  Topic20  -5.9305   0.0867
30    백성     1.566320   203.120743  Topic20  -5.8835  -0.3827
196   부사     1.345729    60.011848  Topic20  -6.0353   0.6847
177  그대로     1.281899   162.750168  Topic20  -6.0839  -0.3615

[1528 rows x 6 columns], token_table=      Topic      Freq Term
term                      
87        1  0.211931   가뭄
87        2  0.105965   가뭄
87        3  0.105965   가뭄
87        4  0.105965   가뭄
87        5  0.035322   가뭄
...     ...       ...  ...
3471      1  0.260477   휘사
3471     14  0.260477   휘사
1881      3  0.546603   흉풍
2293      4  0.368015  희천군
2293     13  0.368015  희천군

[4071 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 1, 4, 14, 7, 16, 5, 3, 6, 19, 8, 11, 20, 17, 18, 9, 12, 2, 15, 10])

In [24]:
# 논의

In [25]:
data =[]
for i in range(0,1):
    txt1 = open('태종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '태종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '논의' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'태종실록'))

In [26]:
len(df)

697

In [27]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("태종실록"," "))
df=df.map(lambda x : x.replace("태종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [28]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','하니'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [29]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [30]:
 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [31]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	모두	0.011911002919077873
	왜인	0.009457452222704887
	요동	0.008278301917016506
	영락	0.007859204895794392
	쇄출하	0.007643651217222214
	주민	0.007632975932210684
	우산	0.0076317936182022095
	논의	0.0071468716487288475
	우리	0.006745453458279371
	재목	0.0063575017265975475


Topic ID: 1
	어찌	0.015357243828475475
	모두	0.014384325593709946
	대간	0.009373096749186516
	때문	0.008267306722700596
	반드시	0.0073320260271430016
	영락	0.006920350715517998
	논의	0.006797080393880606
	비록	0.006092383526265621
	백성	0.006073622964322567
	생각	0.005930420011281967


Topic ID: 2
	의정부	0.008450371213257313
	모두	0.008167277090251446
	논의	0.00807275716215372
	의논	0.008017372339963913
	대답	0.006109667010605335
	진언	0.00604123855009675
	영락	0.0060354359447956085
	백성	0.00558683043345809
	어찌	0.004993851762264967
	태상왕	0.004915203433483839


Topic ID: 3
	모두	0.010925755836069584
	어찌	0.008767757564783096
	백성	0.008043097332119942
	논의	0.007809922564774752
	영락	0.007545433938503265
	대답	0.0068123843520879745
	까닭	0.006129781

In [32]:
# 논의
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.032112 -0.023050       1        1  19.449675
2     -0.006800  0.004774       2        1  11.468029
16    -0.002584 -0.034864       3        1   8.501998
4      0.029465 -0.002513       4        1   7.017606
11    -0.015725  0.097077       5        1   6.267104
6     -0.003231 -0.032800       6        1   5.618483
15     0.022175  0.027273       7        1   5.423862
13     0.041173 -0.011097       8        1   4.859182
7      0.027824  0.008609       9        1   4.501650
17     0.042532 -0.027606      10        1   4.296878
5      0.022661  0.005301      11        1   4.201080
14     0.033820 -0.023429      12        1   3.574490
3     -0.032764 -0.012896      13        1   3.459024
12    -0.029160  0.045589      14        1   3.327076
0     -0.118500 -0.062951      15        1   2.805519
10     0.004477 -0.004026      16        1   2.147336
19    -0.045442  0.063940      17        1   1.714050
8     -0.000589  0.012469      18        1   0.907505
9      0.017593 -0.015097      19        1   0.427401
18    -0.019037 -0.014702      20        1   0.032063, topic_info=     Term        Freq       Total Category  logprob  loglift
31     어찌  110.000000  110.000000  Default  30.0000  30.0000
11     모두  134.000000  134.000000  Default  29.0000  29.0000
38     의논   71.000000   71.000000  Default  28.0000  28.0000
2933   방문   23.000000   23.000000  Default  27.0000  27.0000
108    우리   37.000000   37.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
387    매우    0.006368   22.189045  Topic20  -6.4170  -0.1109
121    가지    0.007160   47.677280  Topic20  -6.2996  -0.7584
95     때문    0.007239   55.150234  Topic20  -6.2887  -0.8931
5      논의    0.006532   72.892014  Topic20  -6.3915  -1.2748
35     영락    0.006286   69.617073  Topic20  -6.4299  -1.2672

[1339 rows x 6 columns], token_table=      Topic      Freq Term
term                      
119       1  0.172088   가게
119       3  0.172088   가게
119       8  0.172088   가게
119      14  0.172088   가게
2571      3  0.224639   가뭄
...     ...       ...  ...
1119      5  0.231465   효종
1119     14  0.462929   효종
85        1  0.119277   흉년
85        3  0.357832   흉년
85        6  0.119277   흉년

[2421 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 17, 5, 12, 7, 16, 14, 8, 18, 6, 15, 4, 13, 1, 11, 20, 9, 10, 19])